# Detecting Auto insurance fraud claims

In [79]:
# Import all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, accuracy_score, recall_score, roc_curve, precision_recall_curve, auc
from sklearn.tree import DecisionTreeClassifier
import warnings

In [80]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_Interactivity = 'all'

In [81]:
# Read dataset
data = pd.read_csv("C:\\Users\\Anju\\Desktop\\Data science misc\\Hand on Projects\\Auto Insurance Fraud Prediction - Streamlit\\insurance_claims.csv")

In [82]:
data.shape

(1000, 40)

In [83]:
# fraud_reported is our target column. We will convert it to 1 and 0 and build the target dataframe.
data['fraud_reported'] = data['fraud_reported'].str.replace('Y', '1')
data['fraud_reported'] = data['fraud_reported'].str.replace('N', '0')
data['fraud_reported'] = data['fraud_reported'].astype(int)
data_target = data['fraud_reported']
data_target.shape

(1000,)

In [105]:
# In another notebook 'Auto Insurance Fraud claim detection-background analysis' where I used techniques like correlation matrix and Chi square 
# for feature selection, below are the important features I have selected to use for my app to predict the fraud claims.

# 'insured_hobbies', 'collision_type',  'months_as_customer_groups', 'policy_deductable', 'incident_severity',
# 'vehicle_claim_groups', 'umbrella_limit', 'number_of_vehicles_involved' ,'bodily_injuries','witnesses', 'incident_type', 
# 'authorities_contacted' ,'police_report_available'

# Reading the dataset with all the above columns:
data_app = pd.read_csv("C:\\Users\\Anju\\Desktop\\Data science misc\\Hand on Projects\\Auto Insurance Fraud Prediction - Streamlit\\data_req.csv")
data_app.head()   # It has only the input features.


,insured_hobbies_new,collision_type_new,months_as_customer_groups,policy_deductable,incident_severity,vehicle_claim_groups,umbrella_limit,number_of_vehicles_involved,bodily_injuries,witnesses,incident_type,authorities_contacted,police_report_available_new
0,other,Side Collision,301-350,1000,Major Damage,50001-60000,0,1,1,2,Single Vehicle Collision,Police,YES
1,other,other,201-250,2000,Minor Damage,0-10000,5000000,1,0,0,Vehicle Theft,Police,other
2,other,Rear Collision,101-150,2000,Minor Damage,20001-30000,5000000,3,2,3,Multi-vehicle Collision,Police,NO
3,other,Front Collision,251-300,2000,Major Damage,50001-60000,6000000,1,1,2,Single Vehicle Collision,Police,NO
4,other,other,201-250,1000,Minor Damage,0-10000,6000000,1,0,1,Vehicle Theft,None,NO


In [86]:
data_app.shape

(1000, 13)

# label encoding

In [87]:
data_app.dtypes    

insured_hobbies_new            object
collision_type_new             object
months_as_customer_groups      object
policy_deductable               int64
incident_severity              object
vehicle_claim_groups           object
umbrella_limit                  int64
number_of_vehicles_involved     int64
bodily_injuries                 int64
witnesses                       int64
incident_type                  object
authorities_contacted          object
police_report_available_new    object
dtype: object

In [88]:
# label endcoding for the object datatypes.

for col in ['insured_hobbies_new', 'collision_type_new', 'months_as_customer_groups', 'incident_severity',
'vehicle_claim_groups', 'incident_type', 'authorities_contacted' ,'police_report_available_new']:
    if (data_app[col].dtype == 'object'):
        le = preprocessing.LabelEncoder()
        le = le.fit(data_app[col])
        data_app[col] = le.transform(data_app[col])
        print('Completed Label encoding on',col)


Completed Label encoding on insured_hobbies_new
Completed Label encoding on collision_type_new
Completed Label encoding on months_as_customer_groups
Completed Label encoding on incident_severity
Completed Label encoding on vehicle_claim_groups
Completed Label encoding on incident_type
Completed Label encoding on authorities_contacted
Completed Label encoding on police_report_available_new


In [89]:
data_app.head()

,insured_hobbies_new,collision_type_new,months_as_customer_groups,policy_deductable,incident_severity,vehicle_claim_groups,umbrella_limit,number_of_vehicles_involved,bodily_injuries,witnesses,incident_type,authorities_contacted,police_report_available_new
0,2,2,5,1000,0,5,0,1,1,2,2,4,1
1,2,3,3,2000,1,0,5000000,1,0,0,3,4,2
2,2,1,1,2000,1,2,5000000,3,2,3,0,4,0
3,2,0,4,2000,0,5,6000000,1,1,2,2,4,0
4,2,3,3,1000,1,0,6000000,1,0,1,3,2,0


In [90]:
# Feature Scaling.

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
data_scaled = sc.fit_transform(data_app)


In [91]:
# Splitting data into train and test

x_train, x_test, y_train, y_test = train_test_split(data_scaled, data_target, random_state = 1)
print('x_train:', x_train.shape, 'x_test:', x_test.shape, 'y_train:', y_train.shape, 'y_test:', y_test.shape )


x_train: (750, 13) x_test: (250, 13) y_train: (750,) y_test: (250,)


# Logistic regression

In [92]:
log = LogisticRegression()

log.fit(x_train, y_train)
prediction = log.predict(x_test)

score = log.score(x_test, y_test)
print(score*100)
print()
print(classification_report(y_test, prediction))


82.8

              precision    recall  f1-score   support

           0       0.83      0.96      0.89       180
           1       0.83      0.49      0.61        70

    accuracy                           0.83       250
   macro avg       0.83      0.72      0.75       250
weighted avg       0.83      0.83      0.81       250



# Decision Tree

In [93]:
dtc = DecisionTreeClassifier()

dtc.fit(x_train, y_train)
preds = dtc.predict(x_test)

score = dtc.score(x_test, y_test)
print(score*100)
print()
print(classification_report(y_test, preds))

78.8

              precision    recall  f1-score   support

           0       0.82      0.91      0.86       180
           1       0.67      0.47      0.55        70

    accuracy                           0.79       250
   macro avg       0.74      0.69      0.71       250
weighted avg       0.78      0.79      0.78       250



# Random Forest

In [94]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1)
rfc.fit(x_train, y_train)
preds = rfc.predict(x_test)

score = rfc.score(x_test, y_test)
print(score*100)
print()
print(classification_report(y_test, preds))

82.39999999999999

              precision    recall  f1-score   support

           0       0.85      0.92      0.88       180
           1       0.73      0.59      0.65        70

    accuracy                           0.82       250
   macro avg       0.79      0.75      0.77       250
weighted avg       0.82      0.82      0.82       250



# Random Forest with grid search CV

In [95]:
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
parameters = {'n_estimators':[100,300],'n_jobs':[-1], 'max_features': [0.5,0.7,0.9], 'min_samples_split': [2, 5, 10, 15],'max_depth': [3,5,7,15],'min_samples_leaf':[1,2,5,10],'random_state':[14]} 

clf1 = GridSearchCV(RandomForestClassifier(), parameters, cv=5, scoring='roc_auc')
clf1.fit(x_train, y_train)
pred_tuned = clf1.predict(x_test)

score = clf1.score(x_test, y_test)
print(score*100)
print()
print(classification_report(y_test, preds))


90.65873015873017

              precision    recall  f1-score   support

           0       0.85      0.92      0.88       180
           1       0.73      0.59      0.65        70

    accuracy                           0.82       250
   macro avg       0.79      0.75      0.77       250
weighted avg       0.82      0.82      0.82       250



In [96]:
# predictions on test using clf model.
pred_tuned = clf1.predict(x_test)
pred_tuned.shape

(250,)

In [106]:
# Creating dataframe with actual and predicted values to compare.

diff = pd.DataFrame({'Actual': y_test, 'Predicted': pred_tuned})
diff.head()


,Actual,Predicted
507,0,0
818,0,0
452,0,0
368,1,0
242,0,0


In [107]:
# Adding new columns 'actual' and 'predictions' to data_app in order to understand what type of records are predicted correctly.

data_app['actual'] = diff['Actual']
data_app['predictions'] = diff['Predicted']


In [108]:
# In below result, we will see many actual and predictions as NaN because data_app consists of 1000 rows and with my model, I
# predicted on only 250 rows.

data_app


,insured_hobbies_new,collision_type_new,months_as_customer_groups,policy_deductable,incident_severity,vehicle_claim_groups,umbrella_limit,number_of_vehicles_involved,bodily_injuries,witnesses,incident_type,authorities_contacted,police_report_available_new,actual,predictions
0,other,Side Collision,301-350,1000,Major Damage,50001-60000,0,1,1,2,Single Vehicle Collision,Police,YES,NaN,NaN
1,other,other,201-250,2000,Minor Damage,0-10000,5000000,1,0,0,Vehicle Theft,Police,other,NaN,NaN
2,other,Rear Collision,101-150,2000,Minor Damage,20001-30000,5000000,3,2,3,Multi-vehicle Collision,Police,NO,0.0,0.0
3,other,Front Collision,251-300,2000,Major Damage,50001-60000,6000000,1,1,2,Single Vehicle Collision,Police,NO,1.0,1.0
4,other,other,201-250,1000,Minor Damage,0-10000,6000000,1,0,1,Vehicle Theft,None,NO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,other,Front Collision,0-50,1000,Minor Damage,60001-70000,0,1,0,1,Single Vehicle Collision,Fire,other,0.0,0.0
996,other,Rear Collision,251-300,1000,Major Damage,70001-80000,0,1,2,3,Single Vehicle Collision,Fire,other,NaN,NaN
997,other,Side Collision,101-150,500,Minor Damage,50001-60000,3000000,3,2,3,Multi-vehicle Collision,Police,YES,0.0,0.0
998,other,Rear Collision,451-500,2000,Major Damage,30001-40000,5000000,1,0,1,Single Vehicle Collision,Other,YES,NaN,NaN


In [130]:
# Dropping all the rows with NaN values.
# So that I can have a cleaned view of my predictions.

data_app_prediction = data_app.dropna()
print(data_app_prediction.shape)
data_app_prediction.head()



(250, 15)


,insured_hobbies_new,collision_type_new,months_as_customer_groups,policy_deductable,incident_severity,vehicle_claim_groups,umbrella_limit,number_of_vehicles_involved,bodily_injuries,witnesses,incident_type,authorities_contacted,police_report_available_new,actual,predictions
2,other,Rear Collision,101-150,2000,Minor Damage,20001-30000,5000000,3,2,3,Multi-vehicle Collision,Police,NO,0.0,0.0
3,other,Front Collision,251-300,2000,Major Damage,50001-60000,6000000,1,1,2,Single Vehicle Collision,Police,NO,1.0,1.0
6,other,Front Collision,101-150,1000,Minor Damage,50001-60000,0,3,0,0,Multi-vehicle Collision,Police,other,0.0,0.0
8,other,Front Collision,0-50,500,Total Loss,20001-30000,0,1,1,1,Single Vehicle Collision,Police,YES,0.0,0.0
12,other,Rear Collision,51-100,500,Total Loss,40001-50000,3000000,1,1,0,Single Vehicle Collision,Ambulance,NO,0.0,0.0


# With Grid search CV applied on random Forest model, accuracy increased to 90.6%, hence we will be using random forest clf1 as our final model and deploy it to production to predict Auto insurance fraud claim detection.

In [ ]:
# Finally save model using pickle and build app uisng SteamLit

In [131]:
import streamlit as st
import pickle

# save the model
filename = 'Streamlit_Autoinsurancefraud.sav'
pickle.dump(clf1, open(filename,'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)


0.9065873015873016


In [ ]:
# Now we will use another notebook named 'Predict Auto Insurance Fraud(streamlit)' 
# to build the app to predict the whether a reported claim is fraud or genuine using streamlit and again save the notebook in 
# C folder with name 'streamlitpredictautoinsurancefraud'